This code is adapted from:
https://github.com/apple/turicreate/blob/master/userguide/object_detection/data-preparation.md 

It serves to convert the csv bounding boxes to csv format which turicreate can use.

In [11]:
import turicreate as tc
import os

IMAGES_DIR = 'cropped_plates' # Change if applicable
csv_path = 'out.csv' # assumes CSV column format is image,id,name,xMin,xMax,yMin,yMax
csv_sf = tc.SFrame.read_csv(csv_path)

print(csv_sf)

def row_to_bbox_coordinates(row):
    """
    Takes a row and returns a dictionary representing bounding
    box coordinates:  (center_x, center_y, width, height)  e.g. {'x': 100, 'y': 120, 'width': 80, 'height': 120}
    """
    return {'x': row['xmin'] + (row['xmax'] - row['xmin'])/2, 
            'width': (row['xmax'] - row['xmin']),
            'y': row['ymin'] + (row['ymax'] - row['ymin'])/2, 
            'height': (row['ymax'] - row['ymin'])}

csv_sf['coordinates'] = csv_sf.apply(row_to_bbox_coordinates)

# delete no longer needed columns
del csv_sf['xmin'], csv_sf['xmax'], csv_sf['ymin'], csv_sf['ymax']

print(csv_sf)

# rename columns
csv_sf = csv_sf.rename({'name': 'label'})
csv_sf = csv_sf.rename({'image': 'name'})

print(csv_sf)

# Load all images in random order
sf_images = tc.image_analysis.load_images(IMAGES_DIR, recursive=True,
    random_order=True)

# Split path to get filename
info = sf_images['path'].apply(lambda path: os.path.basename(path).split('/')[:1])

# Rename columns to 'name'
info = info.unpack().rename({'X.0': 'name'})

# Add to our main SFrame
sf_images = sf_images.add_columns(info)

# Original path no longer needed
del sf_images['path']

# Combine label and coordinates into a bounding box dictionary
csv_sf = csv_sf.pack_columns(['label', 'coordinates'], new_column_name='bbox', dtype=dict)

# Combine bounding boxes of the same 'name' into lists
sf_annotations = csv_sf.groupby('name', 
    {'annotations': tc.aggregate.CONCAT('bbox')})

# Join annotations with the images. Note, some images do not have annotations,
# but we still want to keep them in the dataset. This is why it is important to
# a LEFT join.
sf = sf_images.join(sf_annotations, on='name', how='left')

# The LEFT join fills missing matches with None, so we replace these with empty
# lists instead using fillna.
sf['annotations'] = sf['annotations'].fillna([])

# Save SFrame
sf.save('ig02_plates.sframe')

Finished parsing file /Users/gretakintzley/Google Drive/Senior Year/Senior Design/out.csv

Parsing completed. Parsed 95 lines in 0.035942 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,int,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/gretakintzley/Google Drive/Senior Year/Senior Design/out.csv

Parsing completed. Parsed 95 lines in 0.014178 secs.

+----------+---------------+------+------+------+------+
|  image   |      name     | xmin | xmax | ymin | ymax |
+----------+---------------+------+------+------+------+
| img0.jpg | License Plate | 258  | 387  | 162  | 228  |
| img1.jpg | License Plate | 213  | 317  | 155  | 216  |
| img2.jpg | License Plate | 230  | 353  | 165  | 248  |
| img3.jpg | License Plate | 274  | 443  | 127  | 223  |
| img4.jpg | License Plate | 254  | 381  | 171  | 238  |
| img5.jpg | License Plate | 282  | 358  | 223  | 265  |
| img6.jpg | License Plate | 284  | 367  | 212  | 252  |
| img7.jpg | License Plate |  73  | 320  | 126  | 254  |
| img8.jpg | License Plate | 217  | 383  | 304  | 390  |
| img9.jpg | License Plate | 226  | 456  | 118  | 223  |
+----------+---------------+------+------+------+------+
[95 rows x 6 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
+----------+---------------+------------------------

In [12]:
type(sf)

turicreate.data_structures.sframe.SFrame